In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
data = pd.read_csv('./gossipTimeSeries.csv', header=0, index_col=0, low_memory=False)
gossip = pd.DataFrame(data)

gossip.head(10)

,_time,_measurement,identifier,experiment
0.0,2023-10-04 13:38:40.931335+00:00,iwant,3,385
1.0,2023-10-04 13:38:50.042079+00:00,iwant,3,385
2.0,2023-10-04 13:38:55.094402+00:00,iwant,3,385
3.0,2023-10-04 13:38:55.098665+00:00,iwant,3,385
4.0,2023-10-04 13:39:19.757239+00:00,iwant,3,385
5.0,2023-10-04 13:39:44.211931+00:00,iwant,3,385
6.0,2023-10-04 13:39:44.212174+00:00,iwant,3,385
7.0,2023-10-04 13:39:44.213046+00:00,iwant,3,385
8.0,2023-10-04 13:39:44.213272+00:00,iwant,3,385
9.0,2023-10-04 13:39:44.213613+00:00,iwant,3,385


In [3]:
gossip['type'] = "16.0"
gossip.loc[gossip['_measurement'] == 'ihave', 'type'] = "17.0"
gossip = gossip.drop(columns=['_measurement'])

gossip.head(10)

,_time,identifier,experiment,type
0.0,2023-10-04 13:38:40.931335+00:00,3,385,16.0
1.0,2023-10-04 13:38:50.042079+00:00,3,385,16.0
2.0,2023-10-04 13:38:55.094402+00:00,3,385,16.0
3.0,2023-10-04 13:38:55.098665+00:00,3,385,16.0
4.0,2023-10-04 13:39:19.757239+00:00,3,385,16.0
5.0,2023-10-04 13:39:44.211931+00:00,3,385,16.0
6.0,2023-10-04 13:39:44.212174+00:00,3,385,16.0
7.0,2023-10-04 13:39:44.213046+00:00,3,385,16.0
8.0,2023-10-04 13:39:44.213272+00:00,3,385,16.0
9.0,2023-10-04 13:39:44.213613+00:00,3,385,16.0


In [4]:
data = pd.read_csv('./messageTimeSeries.csv', header=0, index_col=0, low_memory=False)
message = pd.DataFrame(data)

message = message.drop(columns=['start', 'end'])

message.head(10)

,_time,type,identifier,experiment
0.0,2023-10-04 13:38:35.589925+00:00,11.0,3,385
1.0,2023-10-04 13:38:35.589937+00:00,11.0,3,385
2.0,2023-10-04 13:38:35.589941+00:00,11.0,3,385
3.0,2023-10-04 13:38:35.589966+00:00,11.0,3,385
4.0,2023-10-04 13:38:35.590403+00:00,11.0,3,385
5.0,2023-10-04 13:38:35.590406+00:00,11.0,3,385
6.0,2023-10-04 13:38:35.590583+00:00,11.0,3,385
7.0,2023-10-04 13:38:35.590589+00:00,11.0,3,385
8.0,2023-10-04 13:38:35.632920+00:00,11.0,3,385
9.0,2023-10-04 13:38:35.632939+00:00,11.0,3,385


In [5]:
df = pd.concat([message, gossip])


del message
del gossip
gc.collect()

0

In [6]:
df.tail(10)

,_time,type,identifier,experiment
5.0,2023-11-22 02:43:27.243782+00:00,17.0,15,662
6.0,2023-11-22 02:43:27.250611+00:00,17.0,15,662
7.0,2023-11-22 02:43:27.251536+00:00,17.0,15,662
8.0,2023-11-22 02:43:27.254099+00:00,17.0,15,662
9.0,2023-11-22 02:43:27.254269+00:00,17.0,15,662
10.0,2023-11-22 02:43:27.255804+00:00,17.0,15,662
11.0,2023-11-22 02:43:27.255816+00:00,17.0,15,662
0.0,2023-11-22 03:16:59.349471+00:00,16.0,16,663
0.0,2023-11-22 03:16:59.346725+00:00,17.0,16,663
1.0,2023-11-22 03:16:59.349417+00:00,17.0,16,663


In [7]:
df.to_csv('./timeSeriesExperiment.csv')

In [8]:
dfNoIndex = df.reset_index()
dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"],format="mixed", infer_datetime_format=True)
# dfNoIndex.head(10)

by_time = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='10s')])['index'].count().reset_index()
dfAggTime = by_time.rename(columns={'index': "count"})

dfAggTime.tail(10)

/tmp/ipykernel_14868/4133272780.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"],format="mixed", infer_datetime_format=True)


DateParseError: Unknown datetime string format, unable to parse: _time, at position 96582372

In [ ]:
dfAggTime.to_csv('./timeSeries10s.csv')

In [7]:
df = df.dropna()
dfNoIndex = df.reset_index()
dfNoIndex = dfNoIndex.drop(columns=['index'])

dfNoIndex = dfNoIndex.drop(dfNoIndex[dfNoIndex['_time'] == '_time'].index)#, inplace = True)

dfNoIndex.loc[dfNoIndex.index >= 203205548].head(10)
# dfNoIndex.head(10)

,_time,type,identifier,experiment
203205548,2023-11-10 23:44:33.386640+00:00,2.0,93,616
203205549,2023-11-10 23:44:33.386812+00:00,2.0,93,616
203205550,2023-11-10 23:44:33.386850+00:00,2.0,93,616
203205551,2023-11-10 23:44:33.386906+00:00,2.0,93,616
203205552,2023-11-10 23:44:33.386947+00:00,2.0,93,616
203205554,2023-11-16 21:11:58.073180+00:00,3.0,0,618
203205555,2023-11-16 21:11:58.073761+00:00,3.0,0,618
203205556,2023-11-16 21:11:58.073836+00:00,3.0,0,618
203205557,2023-11-16 21:11:58.073897+00:00,3.0,0,618
203205558,2023-11-16 21:11:58.074090+00:00,3.0,0,618


In [8]:
# df = df.dropna()
dfNoIndex = dfNoIndex.reset_index()
dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"], infer_datetime_format=True, format='mixed')

by_time2 = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='1s')])['index'].count().reset_index()
dfAggTime = by_time2.rename(columns={'index': "count"})

dfAggTime.to_csv('./timeSeries1s.csv')
dfAggTime.tail(10)


/tmp/ipykernel_40293/2510124400.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"], infer_datetime_format=True, format='mixed')


,identifier,experiment,type,_time,count
617649,93,616,3.0,2023-11-10 23:44:20+00:00,42
617650,93,616,3.0,2023-11-10 23:44:21+00:00,12
617651,93,616,3.0,2023-11-10 23:44:23+00:00,42
617652,93,616,3.0,2023-11-10 23:44:24+00:00,12
617653,93,616,3.0,2023-11-10 23:44:26+00:00,42
617654,93,616,3.0,2023-11-10 23:44:27+00:00,12
617655,93,616,3.0,2023-11-10 23:44:29+00:00,42
617656,93,616,3.0,2023-11-10 23:44:30+00:00,12
617657,93,616,3.0,2023-11-10 23:44:32+00:00,42
617658,93,616,3.0,2023-11-10 23:44:33+00:00,12


In [13]:
dfNoIndex.tail(10)

,_time,type,identifier,experiment
260771724,2023-11-22 02:43:27.243782+00:00,17.0,15,662
260771725,2023-11-22 02:43:27.250611+00:00,17.0,15,662
260771726,2023-11-22 02:43:27.251536+00:00,17.0,15,662
260771727,2023-11-22 02:43:27.254099+00:00,17.0,15,662
260771728,2023-11-22 02:43:27.254269+00:00,17.0,15,662
260771729,2023-11-22 02:43:27.255804+00:00,17.0,15,662
260771730,2023-11-22 02:43:27.255816+00:00,17.0,15,662
260771731,2023-11-22 03:16:59.349471+00:00,16.0,16,663
260771732,2023-11-22 03:16:59.346725+00:00,17.0,16,663
260771733,2023-11-22 03:16:59.349417+00:00,17.0,16,663


In [9]:
print(dfNoIndex.loc[[203205550]])

                                      _time type identifier experiment
203205550  2023-11-10 23:44:33.386850+00:00  2.0         93        616
